In [1]:
import os
import sqlite3
import re
from datetime import datetime

# Caminho para o banco de dados
DB_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                     'data_warehouse', 'stellar_occultations.db')

import format_data

## 1 - Primeiro passo
Interagir com a Data Base. Buscar objetos e curvas, plotá-las e analisá-las

In [3]:
format_data.get_available_objects()

['1983VD7',
 '1991TF1',
 '1994XM2',
 '1998XA53',
 '1999PY3',
 '1999RU208',
 '1999TA91',
 '1999TB100',
 '1999TG246',
 '1999XR13',
 '1999XW211',
 '2000ET64',
 '2000HT75',
 '2000KJ1',
 '2000OW26',
 '2000SW106',
 '2000TK17',
 '2000WV132',
 '2001HS35',
 '2001US197',
 '2002KX14',
 '2002WC19',
 '2003GG42',
 '2003UP55',
 '2005RM43',
 '2007JJ43',
 '2014EZ51',
 'Abundantia',
 'Academia',
 'Adelheid',
 'Adelinda',
 'Admete',
 'Adorea',
 'Adria',
 'Adzhimushkaj',
 'Aegina',
 'Aegle',
 'Aeternitas',
 'Aethra',
 'Aguntina',
 'Aida',
 'Alauda',
 'Alekto',
 'Alemannia',
 'Aletheia',
 'Alfven',
 'Aline',
 'Alkmene',
 'Alphonsina',
 'Alsatia',
 'Althaea',
 'Amalia',
 'Amalthea',
 'Amata',
 'Ambrosia',
 'Anacostia',
 'Angola',
 'Ani',
 'Antenor',
 'Antigone',
 'Antilochus',
 'Antiope',
 'Antonia',
 'Arctica',
 'Arethusa',
 'Armida',
 'Artemis',
 'Asia',
 'Aspasia',
 'Asplinda',
 'Asterope',
 'Atala',
 'Atalante',
 'Ate',
 'Athamantis',
 'Aulis',
 'Aurelia',
 'Aurora',
 'Ausonia',
 'Austria',
 'Backlunda'

In [4]:
objects = format_data.get_available_objects()

# Demonstração com o primeiro objeto
selected_object = objects[:2]
selected_object

['1983VD7', '1991TF1']

In [5]:
dates = [format_data.get_observation_dates(object) for object in selected_object]
dates

[['2020-11-01'], ['2020-11-01']]

In [6]:
format_data.search_light_curves(selected_object[0], dates[0][0])

[{'observation_id': 1,
  'observer_name': 'JordiMarco',
  'is_positive': True,
  'point_count': 1198}]

In [13]:
lcs = []
for ind_objt in range(len(selected_object)):
    for date in dates[ind_objt]:
        lcs.append(format_data.search_light_curves(selected_object[ind_objt], date))
lcs

[[{'observation_id': 1,
   'observer_name': 'JordiMarco',
   'is_positive': True,
   'point_count': 1198}],
 [{'observation_id': 2,
   'observer_name': 'DHerald',
   'is_positive': True,
   'point_count': 826},
  {'observation_id': 3,
   'observer_name': 'WHanna',
   'is_positive': True,
   'point_count': 281}]]

In [14]:
lcs = [format_data.search_light_curves(selected_object[ind_objt], date) 
       for ind_objt in range(len(selected_object)) 
       for date in dates[ind_objt]]
lcs

[[{'observation_id': 1,
   'observer_name': 'JordiMarco',
   'is_positive': True,
   'point_count': 1198}],
 [{'observation_id': 2,
   'observer_name': 'DHerald',
   'is_positive': True,
   'point_count': 826},
  {'observation_id': 3,
   'observer_name': 'WHanna',
   'is_positive': True,
   'point_count': 281}]]

In [12]:
selected_object

['1983VD7', '1991TF1']

In [ ]:
objects = format_data.get_available_objects()

# Demonstração com o primeiro objeto
selected_object = objects[:2]
print(f"\nDatas disponíveis para {selected_object}:")

dates = [format_data.get_observation_dates(object) for object in selected_object]
for date in dates:
    print(f"- {date} Encontrada")

light_curves = [format_data.search_light_curves(selected_object[ind_objt], date) 
       for ind_objt in range(len(selected_object)) 
       for date in dates[ind_objt]]

for lc in light_curves:
    print(f"- {lc.get('observer_name')} (Positivo: {lc.get('is_positive')}, Pontos: {lc.get('point_count')})")

def process_light_curves_individually(selected_objects, max_curves_per_object=3):
    """
    Processa curvas de luz individualmente, salvando cada uma em um DataFrame
    que pode ser manipulado antes de prosseguir para a próxima.
    
    Args:
        selected_objects: Lista de objetos celestes
        max_curves_per_object: Número máximo de curvas por objeto
    """
    import pandas as pd
    # Contador para acompanhar o progresso
    total_curves_processed = 0
    
    # Itera sobre cada objeto
    for obj_index, object_name in enumerate(selected_objects):
        print(f"\n{'='*60}")
        print(f"PROCESSANDO OBJETO {obj_index + 1}/{len(selected_objects)}: {object_name}")
        print(f"{'='*60}")
        
        # Obtém as datas disponíveis para este objeto
        available_dates = format_data.get_observation_dates(object_name)
        
        if not available_dates:
            print(f"Nenhuma data disponível para {object_name}")
            continue
        
        # Itera sobre cada data disponível
        for date_index, selected_date in enumerate(available_dates):
            print(f"\n--- Data {date_index + 1}/{len(available_dates)}: {selected_date} ---")
            
            # Obtém as curvas para este objeto e data
            curves, observers = format_data.format_light_curve_data(
                max_curves_per_object, 
                object_name, 
                selected_date
            )
            
            if not curves:
                print(f"Nenhuma curva encontrada para {object_name} em {selected_date}")
                continue
            
            # Processa cada curva individualmente
            for curve_index, (curve_data, observer_name) in enumerate(zip(curves, observers)):
                print(f"\nProcessando curva {curve_index + 1}/{len(curves)} - Observador: {observer_name}")
                
                # Cria DataFrame com a série temporal atual (SOBREESCREVE a variável)
                df_current_curve = pd.DataFrame({
                    'Time': curve_data['time'],
                    'Flux': curve_data['flux'],
                    'Flux_Normalized': curve_data['flux_normalized']
                })
                
                # Adiciona metadados como colunas
                df_current_curve['Object'] = object_name
                df_current_curve['Date'] = selected_date
                df_current_curve['Observer'] = observer_name
                df_current_curve['Curve_ID'] = f"{object_name}_{selected_date}_{observer_name}"
                
                # Mostra informações da curva atual
                print(f"  - Pontos de dados: {len(df_current_curve)}")
                print(f"  - Tempo: {df_current_curve['Time'].min():.6f} até {df_current_curve['Time'].max():.6f}")
                print(f"  - Fluxo: {df_current_curve['Flux'].min():.6f} até {df_current_curve['Flux'].max():.6f}")
                
                # =================================================================
                # AQUI VOCÊ PODE MANIPULAR O DATAFRAME df_current_curve
                # Exemplos de manipulações possíveis:
                # =================================================================
                
                # Exemplo 1: Filtrar por intervalo de tempo
                # df_current_curve = df_current_curve[
                #     (df_current_curve['Time'] >= time_start) & 
                #     (df_current_curve['Time'] <= time_end)
                # ]
                
                # Exemplo 2: Aplicar suavização
                # df_current_curve['Flux_Smoothed'] = df_current_curve['Flux_Normalized'].rolling(window=5).mean()
                
                # Exemplo 3: Detectar outliers
                # flux_std = df_current_curve['Flux_Normalized'].std()
                # flux_mean = df_current_curve['Flux_Normalized'].mean()
                # df_current_curve['Is_Outlier'] = abs(df_current_curve['Flux_Normalized'] - flux_mean) > 3 * flux_std
                
                # Exemplo 4: Calcular características estatísticas
                flux_stats = {
                    'min_flux': df_current_curve['Flux_Normalized'].min(),
                    'max_flux': df_current_curve['Flux_Normalized'].max(),
                    'mean_flux': df_current_curve['Flux_Normalized'].mean(),
                    'std_flux': df_current_curve['Flux_Normalized'].std(),
                    'range_flux': df_current_curve['Flux_Normalized'].max() - df_current_curve['Flux_Normalized'].min()
                }
                
                print(f"  - Estatísticas: Min={flux_stats['min_flux']:.4f}, Max={flux_stats['max_flux']:.4f}, "
                      f"Média={flux_stats['mean_flux']:.4f}, Std={flux_stats['std_flux']:.4f}")
                
                # =================================================================
                # AQUI VOCÊ PODE SALVAR OU PROCESSAR MAIS A CURVA
                # =================================================================
                
                # Exemplo: Salvar cada curva em arquivo separado
                # output_filename = f"curve_{object_name}_{selected_date}_{observer_name}.csv"
                # df_current_curve.to_csv(output_filename, index=False)
                # print(f"  - Salvo em: {output_filename}")
                
                # Exemplo: Adicionar à lista de curvas processadas
                # processed_curves.append(df_current_curve.copy())
                
                total_curves_processed += 1
                
                # Pausa para visualização (opcional)
                # input("Pressione Enter para continuar para a próxima curva...")
    
    print(f"\n{'='*60}")
    print(f"PROCESSAMENTO CONCLUÍDO!")
    print(f"Total de curvas processadas: {total_curves_processed}")
    print(f"{'='*60}")

In [15]:
selected_object

['1983VD7', '1991TF1']

In [ ]:
curves

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import cm
import seaborn as sns

def plot_light_curves(curves, observers, object_name, date, save_plots=False):
    """
    Plota as curvas de luz de ocultações estelares.
    
    Args:
        curves (list): Lista de dicionários contendo dados das curvas de luz
        observers (list): Lista com nomes dos observadores correspondentes
        object_name (str): Nome do objeto celeste
        date (str): Data da observação
        save_plots (bool): Se True, salva os plots como arquivos PNG
    """
    
    if not curves:
        print("Nenhuma curva para plotar")
        return
    
    # Configuração do estilo
    plt.style.use('seaborn-v0_8')
    sns.set_palette("husl")
    
    # Cores diferentes para cada observador
    colors = cm.tab10(np.linspace(0, 1, len(curves)))
    
    # Figura 1: Todas as curvas normalizadas em um só gráfico
    fig1, ax1 = plt.subplots(figsize=(14, 8))
    
    for i, (curve, observer) in enumerate(zip(curves, observers)):
        time = curve["time"]
        flux_norm = curve["flux_normalized"]
        
        ax1.plot(time, flux_norm, 'o-', color=colors[i], 
                label=f'{observer} ({len(time)} pontos)', 
                linewidth=2, markersize=3, alpha=0.8)
    
    ax1.set_xlabel('Tempo (horas)', fontsize=12)
    ax1.set_ylabel('Fluxo Normalizado', fontsize=12)
    ax1.set_title(f'Curvas de Luz Normalizadas - {object_name} ({date})', fontsize=14, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # Adiciona linha horizontal em y=1 para referência
    ax1.axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
    
    plt.tight_layout()
    
    if save_plots:
        plt.savefig(f'curvas_normalizadas_{object_name}_{date.replace("-", "")}.png', 
                   dpi=300, bbox_inches='tight')
    
    plt.show()
    
    # Figura 2: Curvas originais vs normalizadas (subplots)
    if len(curves) > 1:
        fig2, axes = plt.subplots(2, 1, figsize=(14, 10))
        
        # Subplot 1: Curvas originais
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            time = curve["time"]
            flux = curve["flux"]
            
            axes[0].plot(time, flux, 'o-', color=colors[i], 
                        label=f'{observer}', linewidth=2, markersize=3, alpha=0.8)
        
        axes[0].set_xlabel('Tempo (horas)', fontsize=12)
        axes[0].set_ylabel('Fluxo Original', fontsize=12)
        axes[0].set_title(f'Curvas de Luz Originais - {object_name} ({date})', fontsize=14, fontweight='bold')
        axes[0].grid(True, alpha=0.3)
        axes[0].legend()
        
        # Subplot 2: Curvas normalizadas
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            time = curve["time"]
            flux_norm = curve["flux_normalized"]
            
            axes[1].plot(time, flux_norm, 'o-', color=colors[i], 
                        label=f'{observer}', linewidth=2, markersize=3, alpha=0.8)
        
        axes[1].set_xlabel('Tempo (horas)', fontsize=12)
        axes[1].set_ylabel('Fluxo Normalizado', fontsize=12)
        axes[1].set_title(f'Curvas de Luz Normalizadas - {object_name} ({date})', fontsize=14, fontweight='bold')
        axes[1].grid(True, alpha=0.3)
        axes[1].legend()
        axes[1].axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
        
        plt.tight_layout()
        
        if save_plots:
            plt.savefig(f'comparacao_curvas_{object_name}_{date.replace("-", "")}.png', 
                       dpi=300, bbox_inches='tight')
        
        plt.show()
    
    # Figura 3: Curvas individuais (uma por subplot)
    if len(curves) > 1:
        n_curves = len(curves)
        n_cols = 2 if n_curves > 1 else 1
        n_rows = (n_curves + n_cols - 1) // n_cols
        
        fig3, axes = plt.subplots(n_rows, n_cols, figsize=(14, 5*n_rows))
        if n_curves == 1:
            axes = [axes]
        elif n_rows == 1:
            axes = axes.reshape(1, -1)
        
        for i, (curve, observer) in enumerate(zip(curves, observers)):
            row = i // n_cols
            col = i % n_cols
            ax = axes[row, col] if n_rows > 1 else axes[col]
            
            time = curve["time"]
            flux_norm = curve["flux_normalized"]
            
            ax.plot(time, flux_norm, 'o-', color=colors[i], 
                   linewidth=2, markersize=4, alpha=0.8)
            
            ax.set_xlabel('Tempo (horas)', fontsize=10)
            ax.set_ylabel('Fluxo Normalizado', fontsize=10)
            ax.set_title(f'{observer} - {len(time)} pontos', fontsize=12, fontweight='bold')
            ax.grid(True, alpha=0.3)
            ax.axhline(y=1, color='black', linestyle='--', alpha=0.5, linewidth=1)
        
        # Remove subplots vazios
        for i in range(len(curves), n_rows * n_cols):
            row = i // n_cols
            col = i % n_cols
            ax = axes[row, col] if n_rows > 1 else axes[col]
            ax.remove()
        
        plt.suptitle(f'Curvas Individuais - {object_name} ({date})', fontsize=16, fontweight='bold')
        plt.tight_layout()
        
        if save_plots:
            plt.savefig(f'curvas_individuais_{object_name}_{date.replace("-", "")}.png', 
                       dpi=300, bbox_inches='tight')
        
        plt.show()


def plot_statistics(curves, observers, object_name, date):
    """
    Plota estatísticas das curvas de luz.
    
    Args:
        curves (list): Lista de dicionários contendo dados das curvas de luz
        observers (list): Lista com nomes dos observadores correspondentes
        object_name (str): Nome do objeto celeste
        date (str): Data da observação
    """
    
    if not curves:
        print("Nenhuma curva para analisar")
        return
    
    # Calcula estatísticas
    stats = []
    for curve, observer in zip(curves, observers):
        flux_norm = curve["flux_normalized"]
        stats.append({
            'observer': observer,
            'n_points': len(flux_norm),
            'min_flux': min(flux_norm),
            'max_flux': max(flux_norm),
            'mean_flux': np.mean(flux_norm),
            'std_flux': np.std(flux_norm),
            'range_flux': max(flux_norm) - min(flux_norm)
        })
    
    # Gráfico de barras com estatísticas
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    observers_short = [obs[:15] + '...' if len(obs) > 15 else obs for obs in observers]
    
    # Número de pontos
    axes[0, 0].bar(observers_short, [s['n_points'] for s in stats], color='skyblue')
    axes[0, 0].set_title('Número de Pontos por Observador', fontweight='bold')
    axes[0, 0].set_ylabel('Número de Pontos')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    # Amplitude (range) do fluxo
    axes[0, 1].bar(observers_short, [s['range_flux'] for s in stats], color='lightcoral')
    axes[0, 1].set_title('Amplitude do Fluxo Normalizado', fontweight='bold')
    axes[0, 1].set_ylabel('Amplitude (Max - Min)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Desvio padrão
    axes[1, 0].bar(observers_short, [s['std_flux'] for s in stats], color='lightgreen')
    axes[1, 0].set_title('Desvio Padrão do Fluxo', fontweight='bold')
    axes[1, 0].set_ylabel('Desvio Padrão')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # Fluxo médio
    axes[1, 1].bar(observers_short, [s['mean_flux'] for s in stats], color='gold')
    axes[1, 1].set_title('Fluxo Médio Normalizado', fontweight='bold')
    axes[1, 1].set_ylabel('Fluxo Médio')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].axhline(y=1, color='red', linestyle='--', alpha=0.7, linewidth=1)
    
    plt.suptitle(f'Estatísticas das Curvas - {object_name} ({date})', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Imprime tabela de estatísticas
    print(f"\n{'='*80}")
    print(f"ESTATÍSTICAS DAS CURVAS - {object_name} ({date})")
    print(f"{'='*80}")
    print(f"{'Observador':<20} {'N Pontos':<10} {'Min':<8} {'Max':<8} {'Média':<8} {'Desvio':<8} {'Amplitude':<10}")
    print(f"{'-'*80}")
    
    for s in stats:
        print(f"{s['observer']:<20} {s['n_points']:<10} {s['min_flux']:<8.4f} {s['max_flux']:<8.4f} "
              f"{s['mean_flux']:<8.4f} {s['std_flux']:<8.4f} {s['range_flux']:<10.4f}")


# Exemplo de uso:
# Assumindo que você já tem as variáveis curves, observers, object_name e date
# plot_light_curves(curves, observers, object_name, date, save_plots=True)
# plot_statistics(curves, observers, object_name, date)

## 2 - Segundo passo
Ler uma amostragem grande de curvas (+100), programar um método de recortar trechos negativos e associá-los a curvas separadas negativas. 

Ideia: separar recortes em outra variável "fake_negative".

## 3 - Terceiro passo
Organizar variáveis de curvas lidas, aplicar feature engineering e criar DataSet de treino/validação.